# single source shortest path
* label setting algorithm http://www.csie.ntnu.edu.tw/~u91029/Path.html#3
    * $O(V^3)$
* read coordinates from google map http://www.mapcoordinates.net/en

In [1]:
import numpy as np

In [2]:
def find_shortest_path(start_id, end_id, edges):
    '''
    edges: [(ns_1, ne_1, w_1), (ns_2, ne_2, w_2)]
    node labels should be integers starting from 0
    '''
    
    num_nodes = max([max(i,j) for i,j,w in edge_list])+1
    print('num nodes:',num_nodes)
    distance = np.full(num_nodes, np.nan)
    visited = np.full(num_nodes, False) 
    parent = [np.nan] * num_nodes
    # compute
    visited[start_id] = True
    distance[start_id] = 0
    while len(edges) > 0:
        min_d = np.inf # big number
        min_edge = None
        for edge in edges:
            i, j, w = edge
            if visited[i] and (not visited[j]):
                if w + distance[i] < min_d:
                    min_d = w + distance[i]
                    min_edge = edge
        if min_edge is None:
            break
        i, j, w = min_edge
        parent[j] = i
        visited[j] = True
        distance[j] = distance[i] + w
        edges.remove(min_edge)

    shortest_path =[end_id]
    if np.isnan(parent[end_id]):
        shortest_path = None
    else:
        while shortest_path[0] != start_id:
#             print(parent[shortest_path[0]])
            shortest_path.insert(0, parent[shortest_path[0]])

    return shortest_path, distance, parent

## example problem (num_nodes = 9)

In [3]:

edge_list = [(0,1,5), (0,5,2), (0,3,3), 
         (1,4,3), 
         (2,4,1),
         (3,6,5),
         (4,8,5),
         (5,0,1), (5,4,0),
         (6,7,3),
         (7,5,1),
         (8,5,2), (8,7,4), (8,8,2)
        ] 
start_id = 0
end_id = 8
shortest_path, distance, parent = find_shortest_path(start_id, end_id, edge_list)
print('Shortest path from', start_id, 'to', end_id, 'is', shortest_path)

num nodes: 9
Shortest path from 0 to 8 is [0, 5, 4, 8]


## google map example

In [4]:
# ---- map data ------

edge_list0 = [(0,1),  (1,2),  (0,3),  (1,4),  (2,5),  (3,4),  (4,5),  (3,6),  (4,7),  (5,8),
              (6,7),  (7,8),  (6,9),  (7,10), (8,13), (9,10), (9,14), (14,15),(11,15),(10,11),
              (11,12),(12,13),(11,16),(12,17),(13,18),(15,16),(16,17),(17,18),(14,19),(15,20),
              (17,21),(18,22),(19,20),(20,21),(21,22)
        ] 
num_edges0 = len(edge_list0)
is_park0 = np.full(num_edges0, False)
is_park0[[10, 12, 13, 15, 18, 22, 25, 20, 23, 26]] = True
is_bike0 = np.full(num_edges0, False)
is_bike0[[3,7,8,10,11,12,13,16,19,22,23,25,26,27,29,31,33,34]] = True
is_crowded0 = np.full(num_edges0, False)
is_crowded0[[2,9,28]] = True


# crowd0 = np.full(num_edges0, 0.0)
# crowd0[2] = 0.5
# crowd0[9] = 0.7
# crowd0[28] = 0.6 

lat_lon = [(25.04154273, 121.54892945), 
           (25.03786839, 121.54888654), 
           (25.03322184, 121.54877925), 
           (25.04142609, 121.55300641),
           (25.03781006, 121.55298495), 
           (25.03322184, 121.55251288),
           (25.0413872,  121.55768418), 
           (25.03765453, 121.5575769),
           (25.03314407, 121.55744815), #8
           (25.04121224, 121.56156802),
           (25.03759621, 121.56148219),
           (25.037499,   121.56208301),
           (25.03586592, 121.56131053), #12 
           (25.03304686, 121.55961537),
           (25.04113448, 121.56444335), # 14
           (25.03919039, 121.56349921), 
           (25.03753789, 121.56358504),
           (25.0359048,  121.56358504),
           (25.03304686, 121.56341338),
           (25.04097895, 121.56858468), # 19
           (25.03901542, 121.56845594), # 20
           (25.03586592, 121.56834865),
           (25.03279411, 121.5683701)
          ]
# -----

def compute_weight(edge_list0, lat_lon, is_bike, is_park, is_crowded, fac_bike=1.0, fac_park=1.0, fac_crowded=1.0):
    edge_list = []
    for i_edge, (p1, p2) in enumerate(edge_list0):
        lat1, lon1 = lat_lon[p1]
        lat2, lon2 = lat_lon[p2]
        w = np.sqrt((lat2-lat1)**2 + (lon2-lon1)**2)
        w0=w
        if is_park0[i_edge]:
            w *= fac_park
        if is_bike0[i_edge]:
            w *= fac_bike
        if is_crowded[i_edge]:
            w *= fac_crowded
#         w = w + fac_crowd*(1+crowd0[i_edge])
        edge_list.append((p1, p2, w))
        edge_list.append((p2, p1, w)) # assume all roads go both way
    return edge_list

# get effective road distance by weight
fac_bike = 0.5 # reduce distance by factor if is_bike (1.0=unchanged)
fac_park = 0.5 # reduce distance by factor if is_park (1.0=unchanged)
fac_crowded = 1.5 # increase distance by factor if is crowded

edge_list = compute_weight(edge_list0, lat_lon, is_bike0, is_park0, is_crowded0, fac_bike, fac_park, fac_crowded)

# -------- examle 2 => 20
start_id = 2
end_id = 20

edge_list = compute_weight(edge_list0, lat_lon, is_bike0, is_park0, is_crowded0)
shortest_path_A, distance, parent = find_shortest_path(start_id, end_id, edge_list)
print('Shortest path from', start_id, 'to', end_id, 'is', shortest_path_A)

edge_list = compute_weight(edge_list0, lat_lon, is_bike0, is_park0, is_crowded0, fac_bike, fac_park, fac_crowded)
shortest_path_B, distance, parent = find_shortest_path(start_id, end_id, edge_list)
print('Shortest path from', start_id, 'to', end_id, 'is', shortest_path_B)


# ------ example 
start_id = 0
end_id = 22

edge_list = compute_weight(edge_list0, lat_lon, is_bike0, is_park0, is_crowded0)
shortest_path_2A, distance, parent = find_shortest_path(start_id, end_id, edge_list)
print('Shortest path from', start_id, 'to', end_id, 'is', shortest_path_2A)

edge_list = compute_weight(edge_list0, lat_lon, is_bike0, is_park0, is_crowded0, fac_bike, fac_park, fac_crowded)
shortest_path_2B, distance, parent = find_shortest_path(start_id, end_id, edge_list)
print('Shortest path from', start_id, 'to', end_id, 'is', shortest_path_2B)




num nodes: 23
Shortest path from 2 to 20 is [2, 5, 8, 13, 12, 11, 15, 20]
num nodes: 23
Shortest path from 2 to 20 is [2, 1, 4, 7, 10, 11, 16, 15, 20]
num nodes: 23
Shortest path from 0 to 22 is [0, 3, 4, 7, 8, 13, 18, 22]
num nodes: 23
Shortest path from 0 to 22 is [0, 1, 4, 7, 10, 11, 16, 17, 18, 22]


## visualization

In [5]:
from gmplot import gmplot
import os

def intermediate_points(lat1, lon1, lat2, lon2, step=0.0005):
    n_points = np.ceil(np.sqrt((lat2-lat1)**2 + (lon2-lon1)**2)/step).astype(int)
    return np.linspace(lat1, lat2, n_points), np.linspace(lon1, lon2, n_points)

def create_gmap_demo(lat_lon, edge_list0, center_lat_lon = (25.037583, 121.555356)):
    gmap = gmplot.GoogleMapPlotter(center_lat_lon[0], center_lat_lon[1], 15)
    # plot node positions
    lat, lon = np.array(lat_lon)[:,0], np.array(lat_lon)[:,1]
    gmap.scatter(lat, lon, marker=False, size=20, color='k')
    # available paths 
    edge_coord = []
    for i_edge, (n1, n2) in enumerate(edge_list0):
        (lat1, lon1), (lat2, lon2) = lat_lon[n1], lat_lon[n2]
        if is_bike0[i_edge] and is_park0[i_edge]:
            color, width = ('purple', 7)
        elif is_bike0[i_edge]:
            color, width = ('b', 5)
        elif is_park0[i_edge]:
            color, width = ('g', 5)
        else:
            color, width = ('gray', 3)
        gmap.plot([lat1, lat2], [lon1, lon2], color=color, edge_width=width)
        if is_crowded0[i_edge]:
            lat_inter, lon_inter = intermediate_points(lat1, lon1, lat2, lon2, step=0.001)
            gmap.scatter(lat_inter, lon_inter, marker=False, size=25, color='g')
    return gmap

def gmap_add_path(gmap, lat_lon, path, color, size, step=0.00025):
    routes = [(path[i], path[i+1]) for i in range(len(path)-1)]
    for n1, n2 in routes:
        (lat1, lon1), (lat2, lon2) = lat_lon[n1], lat_lon[n2]
        lat_inter, lon_inter = intermediate_points(lat1, lon1, lat2, lon2, step=step)
        gmap.scatter(lat_inter, lon_inter, marker=False, size=size, color=color)
#     gmap.scatter(lat_inter, lon_inter, marker=False, size=size, color=color)
    # plot start / end of path
    (lat1, lon1), (lat2, lon2) = lat_lon[path[0]], lat_lon[path[-1]]
    gmap.scatter([lat1, lat2], [lon1, lon2], marker=False, size=size+20, color=color)
    return gmap
    

dpath = 'gmap_html'
# --- example 1
# just map
gmap = create_gmap_demo(lat_lon, edge_list0)
gmap.draw(os.path.join(dpath, 'demo_map.html'))
# map + pathA
gmap = create_gmap_demo(lat_lon, edge_list0)
gmap = gmap_add_path(gmap, lat_lon, shortest_path_A, 'k', 20, step=0.00025)
gmap.draw(os.path.join(dpath, 'demo_map_path_A.html'))
# map + pathB
gmap = create_gmap_demo(lat_lon, edge_list0)
gmap = gmap_add_path(gmap, lat_lon, shortest_path_B, 'r', 30, step=0.00025)
gmap.draw(os.path.join(dpath, 'demo_map_path_B.html'))

# map + both path
gmap = create_gmap_demo(lat_lon, edge_list0)
gmap = gmap_add_path(gmap, lat_lon, shortest_path_A, 'k', 20, step=0.00025)
gmap = gmap_add_path(gmap, lat_lon, shortest_path_B, 'r', 30, step=0.00025)
gmap.draw(os.path.join(dpath, 'demo_map_path_A_B.html'))

# --- example 2
# map + path2A
gmap = create_gmap_demo(lat_lon, edge_list0)
gmap = gmap_add_path(gmap, lat_lon, shortest_path_2A, 'k', 20, step=0.00025)
gmap.draw(os.path.join(dpath, 'demo_map_path_2A.html'))
# map + path2B
gmap = create_gmap_demo(lat_lon, edge_list0)
gmap = gmap_add_path(gmap, lat_lon, shortest_path_2B, 'r', 30, step=0.00025)
gmap.draw(os.path.join(dpath, 'demo_map_path_2B.html'))

# map + both path
gmap = create_gmap_demo(lat_lon, edge_list0)
gmap = gmap_add_path(gmap, lat_lon, shortest_path_2A, 'k', 20, step=0.00025)
gmap = gmap_add_path(gmap, lat_lon, shortest_path_2B, 'r', 30, step=0.00025)
gmap.draw(os.path.join(dpath, 'demo_map_path_2A_2B.html'))

In [6]:
# gmap = gmplot.GoogleMapPlotter(25.037583, 121.555356, 15)
# # plot node positions
# lat, lon = np.array(lat_lon)[:,0], np.array(lat_lon)[:,1]
# gmap.scatter(lat, lon, marker=False, size=20, color='k')
# # paths
# edge_coord = []
# for i_edge, (n1, n2) in enumerate(edge_list0):
#     (lat1, lon1), (lat2, lon2) = lat_lon[n1], lat_lon[n2]
#     if is_bike0[i_edge] and is_park0[i_edge]:
#         color, width = ('purple', 7)
#     elif is_bike0[i_edge]:
#         color, width = ('b', 7)
#     elif is_park0[i_edge]:
#         color, width = ('g', 7)
#     else:
#         color, width = ('gray', 5)
#     gmap.plot([lat1, lat2], [lon1, lon2], color=color, edge_width=width)
#     if is_crowded0[i_edge]:
#         lat_inter, lon_inter = intermediate_points(lat1, lon1, lat2, lon2, step=0.001)
#         gmap.scatter(lat_inter, lon_inter, marker=False, size=20, color='y')

# gmap.draw(os.path.join(dpath, 'demo_map.html'))

# # route A (plain distance)
# routes = [(shortest_path_A[i], shortest_path_A[i+1]) for i in range(len(shortest_path_A)-1)]
# for n1, n2 in routes:
#     (lat1, lon1), (lat2, lon2) = lat_lon[n1], lat_lon[n2]
# #     gmap.plot([lat1, lat2] , [lon1, lon2], color='r', edge_width=3)
#     lat_inter, lon_inter = intermediate_points(lat1, lon1, lat2, lon2, step=0.00025)
#     gmap.scatter(lat_inter, lon_inter, marker=False, size=20, color='b')
    
# # route B (consider weights)
# routes = [(shortest_path_B[i], shortest_path_B[i+1]) for i in range(len(shortest_path_B)-1)]
# for n1, n2 in routes:
#     (lat1, lon1), (lat2, lon2) = lat_lon[n1], lat_lon[n2]
# #     gmap.plot([lat1, lat2] ,[lon1, lon2], color='y', edge_width=3)
#     lat_inter, lon_inter = intermediate_points(lat1, lon1, lat2, lon2, step=0.00025)
#     gmap.scatter(lat_inter, lon_inter, marker=False, size=30, color='r')
    
    
# gmap.draw(os.path.join(dpath, 'demo_map_with_routes.html'))